# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

City_ID      City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
0        0     ancud -41.8697  -73.8203     13.58        78          85   
1        1  santanyi  39.3546    3.1291      9.37        77          36   
2        2  tyukhtet  56.5389   89.3011    -17.52        86          90   
3        3    anadyr  64.7500  177.4833    -12.72        84         100   
4        4   iqaluit  63.7506  -68.5145    -14.15        65          75   

   Wind Speed Country        Date  
0        3.31      CL  1733871732  
1        1.20      ES  1733871733  
2        5.08      RU  1733871734  
3        5.00      RU  1733871735  
4        2.06      CA  1733871736

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df1 = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df1 = narrowed_city_df1.dropna()

# Display sample data
narrowed_city_df1

City_ID       City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
97        97     lompoc  34.6391 -120.4579     23.16        42           0   
167      167   kedougou  12.5579  -12.1743     26.32        53           0   
212      212      young -32.6833  -57.6333     21.57        47           0   
236      236   chabahar  25.2919   60.6430     21.11        54           0   
398      398    salalah  17.0151   54.0924     23.05        43           0   
410      410  ad douiem  14.0012   32.3116     24.86        32           0   

     Wind Speed Country        Date  
97         2.06      US  1733871843  
167        0.00      SN  1733871926  
212        4.02      UY  1733871930  
236        2.26      IR  1733872007  
398        3.60      OM  1733872201  
410        4.45      SD  1733872215

### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df1[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df 

City Country      Lat       Lng  Humidity Hotel Name
97      lompoc      US  34.6391 -120.4579        42           
167   kedougou      SN  12.5579  -12.1743        53           
212      young      UY -32.6833  -57.6333        47           
236   chabahar      IR  25.2919   60.6430        54           
398    salalah      OM  17.0151   54.0924        43           
410  ad douiem      SD  14.0012   32.3116        32

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)


    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lompoc - nearest hotel: Red Roof Inn Lompoc
kedougou - nearest hotel: Auberge Thomas Sankara
young - nearest hotel: Hotel El Galeon
chabahar - nearest hotel: هتل سپیده
salalah - nearest hotel: Muscat International Hotel
ad douiem - nearest hotel: No hotel found


City Country      Lat       Lng  Humidity  \
97      lompoc      US  34.6391 -120.4579        42   
167   kedougou      SN  12.5579  -12.1743        53   
212      young      UY -32.6833  -57.6333        47   
236   chabahar      IR  25.2919   60.6430        54   
398    salalah      OM  17.0151   54.0924        43   
410  ad douiem      SD  14.0012   32.3116        32   

                     Hotel Name  
97          Red Roof Inn Lompoc  
167      Auberge Thomas Sankara  
212             Hotel El Galeon  
236                   هتل سپیده  
398  Muscat International Hotel  
410              No hotel found

In [ ]:
Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
badger - nearest hotel: Clarion Hotel & Suites
ust'-kulom - nearest hotel: No hotel found
emerald - nearest hotel: Emerald Central Hotel
newman - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
laguna - nearest hotel: Holiday Inn Express & Suites
nova vicosa - nearest hotel: Pousada Pontal da Barra
fort myers shores - nearest hotel: Comfort Inn Fort Myers
trinity beach - nearest hotel: Blue Lagoon Resort
racine - nearest hotel: No hotel found
six-fours-les-plages - nearest hotel: Kyriad Direct La Seyne-sur-Mer
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
posto fiscal rolim de moura - nearest hotel: No hotel found
port douglas - nearest hotel: Mantra Aqueous Port Douglas
roswell - nearest hotel: Best Western Plus Hotel
hell-ville - nearest hotel: ABUD HOTEL
sona - nearest hotel: Al Bosco
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
vernal - nearest hotel: Landmark Inn and Suites

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot